In [2]:
import requests
import json
import re
import sys
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup('html.parser')
soup

<html><body><p>html.parser</p></body></html>

In [3]:
# 서울시 25개 구의 법정동 코드 앞 5자리
SEOUL_DISTRICTS = {
    "강남구": "11680",
    "강동구": "11740",
    "강북구": "11305",
    "강서구": "11500",
    "관악구": "11620",
    "광진구": "11215",
    "구로구": "11530",
    "금천구": "11545",
    "노원구": "11350",
    "도봉구": "11320",
    "동대문구": "11230",
    "동작구": "11590",
    "마포구": "11440",
    "서대문구": "11410",
    "서초구": "11650",
    "성동구": "11200",
    "성북구": "11290",
    "송파구": "11710",
    "양천구": "11470",
    "영등포구": "11560",
    "용산구": "11170",
    "은평구": "11380",
    "종로구": "11110",
    "중구": "11140",
    "중랑구": "11260"
}


In [50]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"
SERVICE_KEY = "YPxXvm/3jO5hggkbdJFWnhh8IC3VdMlkRvZHh1pkG8eZxXw12ymm4OAW10urfUVN++TzkurcrZ5Os3Gr9P8Kwg=="
URL = f"{BASE_URL}"

# 서울 구별 데이터 저장할 딕셔너리
seoul_real_estate_data = {}

# 크롤링할 기간 설정
start_year = 2025
start_month = 1
end_year = 2025
end_month = 2

# 서울시 각 구별 데이터 요청
# for 서울, 인천, 경기 
# for 연월

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        if year == start_year and month < start_month:
            continue
        if year == end_year and month > end_month:
            break

        deal_ymd = f"{year}{month:02}"  # 거래 연월 (YYYYMM 형식)
        print(f"⏳ {deal_ymd} 데이터 수집 시작")
    
    # 각 구별 데이터 요청       
    for gu, code in SEOUL_DISTRICTS.items():
        print(f"📌 {gu} ({code}) 데이터 요청 중...")

        params = {
            "LAWD_CD": code,
            "DEAL_YMD": deal_ymd,  # 거래 연월
            "serviceKey": SERVICE_KEY,
            "pageNo": 1,
            "numOfRows": 999
        }

        response = requests.get(URL, params=params)
        response.encoding = "utf-8"
        response.text
        
        # 응답 상태 코드 체크
        if response.status_code == 200:
            print(f"✅ {gu} 응답 상태: {response.status_code}")
            soup = BeautifulSoup(response.text, "xml")

            # 응답 XML 구조 확인
            print(f"📊 {gu} 응답 데이터 샘플:")
            print(soup.prettify()[:1000])  # 데이터 일부 출력해서 확인

            # 'item' 태그가 있는지 확인
            items = soup.find_all("item")
            if not items:
                print(f"⚠️ {gu}에 거래 데이터가 없습니다.")
            else:
                seoul_real_estate_data[gu] = items
                print(f"✅ {gu} 데이터 수집 완료 ({len(items)}건)")

                # 데이터 추출 및 검증
                all_data = []  # 구별 데이터 리스트 초기화
                for item in items:
                    data = {
                        "구": gu,
                        "거래금액": item.find("거래금액").text.strip() if item.find("거래금액") else None,
                        "건축년도": item.find("건축년도").text.strip() if item.find("건축년도") else None,
                        "법정동": item.find("법정동").text.strip() if item.find("법정동") else None,
                        "아파트": item.find("아파트").text.strip() if item.find("아파트") else None,
                        "전용면적": item.find("전용면적").text.strip() if item.find("전용면적") else None,
                        "층": item.find("층").text.strip() if item.find("층") else None,
                        "거래일": f"{item.find('년').text}-{item.find('월').text}-{item.find('일').text}"
                        if item.find('년') and item.find('월') and item.find('일') else None
                    }

                    # 데이터 유효성 검사
                    if None not in data.values():  # 모든 값이 유효한 경우에만 리스트에 추가
                        all_data.append(data)

                # DataFrame 변환
                if all_data:
                    df = pd.DataFrame(all_data)
                    print(f"✅ {gu} DataFrame 생성 완료")
                    print(df.head())

        else:
            print(f"❌ {gu} 요청 실패 (Status Code: {response.status_code})")
        
        time.sleep(0.5)  # API 요청 간격 조절

print("📦 데이터 수집 완료")


⏳ 202501 데이터 수집 시작
⏳ 202502 데이터 수집 시작
📌 강남구 (11680) 데이터 요청 중...
✅ 강남구 응답 상태: 200
📊 강남구 응답 데이터 샘플:
<?xml version="1.0" encoding="utf-8"?>
<response>
 <header>
  <resultCode>
   000
  </resultCode>
  <resultMsg>
   OK
  </resultMsg>
 </header>
 <body>
  <items>
   <item>
    <aptDong>
    </aptDong>
    <aptNm>
     개나리래미안
    </aptNm>
    <aptSeq>
     11680-3680
    </aptSeq>
    <bonbun>
     0754
    </bonbun>
    <bubun>
     0000
    </bubun>
    <buildYear>
     2006
    </buildYear>
    <buyerGbn>
     개인
    </buyerGbn>
    <cdealDay>
    </cdealDay>
    <cdealType>
    </cdealType>
    <dealAmount>
     322,000
    </dealAmount>
    <dealDay>
     19
    </dealDay>
    <dealMonth>
     2
    </dealMonth>
    <dealYear>
     2025
    </dealYear>
    <dealingGbn>
     중개거래
    </dealingGbn>
    <estateAgentSggNm>
     서울 강남구
    </estateAgentSggNm>
    <excluUseAr>
     129.8
    </excluUseAr>
    <floor>
     15
    </floor>
    <jibun>
     754
    </jibun>
    <landCd>
     1


In [51]:
import pandas as pd
import xml.etree.ElementTree as ET

df_seoul_real_estate_data = pd.DataFrame()

for gu, items in seoul_real_estate_data.items():
    for item in items:
        # XML 데이터를 파싱하여 Element 객체로 변환
        xml_data = str(item)

        # XML 파싱
        root = ET.fromstring(xml_data)

        # 각 항목을 딕셔너리 형태로 추출하여 리스트에 저장
        data = {
            'aptDong': root.find('aptDong').text if root.find('aptDong') is not None else '',
            'aptNm': root.find('aptNm').text if root.find('aptNm') is not None else '',
            'aptSeq': root.find('aptSeq').text if root.find('aptSeq') is not None else '',
            'bonbun': root.find('bonbun').text if root.find('bonbun') is not None else '',
            'bubun': root.find('bubun').text if root.find('bubun') is not None else '',
            'buildYear': root.find('buildYear').text if root.find('buildYear') is not None else '',
            'buyerGbn': root.find('buyerGbn').text if root.find('buyerGbn') is not None else '',
            'cdealDay': root.find('cdealDay').text if root.find('cdealDay') is not None else '',
            'cdealType': root.find('cdealType').text if root.find('cdealType') is not None else '',
            'dealAmount': root.find('dealAmount').text if root.find('dealAmount') is not None else '',
            'dealDay': root.find('dealDay').text if root.find('dealDay') is not None else '',
            'dealMonth': root.find('dealMonth').text if root.find('dealMonth') is not None else '',
            'dealYear': root.find('dealYear').text if root.find('dealYear') is not None else '',
            'dealingGbn': root.find('dealingGbn').text if root.find('dealingGbn') is not None else '',
            'estateAgentSggNm': root.find('estateAgentSggNm').text if root.find('estateAgentSggNm') is not None else '',
            'excluUseAr': root.find('excluUseAr').text if root.find('excluUseAr') is not None else '',
            'floor': root.find('floor').text if root.find('floor') is not None else '',
            'jibun': root.find('jibun').text if root.find('jibun') is not None else '',
            'landCd': root.find('landCd').text if root.find('landCd') is not None else '',
            'landLeaseholdGbn': root.find('landLeaseholdGbn').text if root.find('landLeaseholdGbn') is not None else '',
            'rgstDate': root.find('rgstDate').text if root.find('rgstDate') is not None else '',
            'roadNm': root.find('roadNm').text if root.find('roadNm') is not None else '',
            'roadNmBonbun': root.find('roadNmBonbun').text if root.find('roadNmBonbun') is not None else '',
            'roadNmBubun': root.find('roadNmBubun').text if root.find('roadNmBubun') is not None else '',
            'roadNmCd': root.find('roadNmCd').text if root.find('roadNmCd') is not None else '',
            'roadNmSeq': root.find('roadNmSeq').text if root.find('roadNmSeq') is not None else '',
            'roadNmSggCd': root.find('roadNmSggCd').text if root.find('roadNmSggCd') is not None else '',
            'roadNmbCd': root.find('roadNmbCd').text if root.find('roadNmbCd') is not None else '',
            'sggCd': root.find('sggCd').text if root.find('sggCd') is not None else '',
            'slerGbn': root.find('slerGbn').text if root.find('slerGbn') is not None else '',
            'umdCd': root.find('umdCd').text if root.find('umdCd') is not None else '',
            'umdNm': root.find('umdNm').text if root.find('umdNm') is not None else ''
        }

        # 데이터프레임으로 변환
        df = pd.DataFrame([data])

        # 기존 데이터프레임에 합침
        df_seoul_real_estate_data = pd.concat([df_seoul_real_estate_data, df], ignore_index=True)

# 결과 출력
df_seoul_real_estate_data

,aptDong,aptNm,aptSeq,bonbun,bubun,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,...,roadNmBonbun,roadNmBubun,roadNmCd,roadNmSeq,roadNmSggCd,roadNmbCd,sggCd,slerGbn,umdCd,umdNm
0,,개나리래미안,11680-3680,0754,0000,2006,개인,,,"322,000",...,00306,00000,3122008,01,11680,0,11680,개인,10100,역삼동
1,,개포2차현대아파트(220),11680-371,0655,0002,1988,개인,,,"136,500",...,00103,00000,3005086,02,11680,0,11680,개인,10300,개포동
2,,대치삼성,11680-205,1014,0003,2000,개인,,,"290,000",...,00022,00000,4166245,01,11680,0,11680,개인,10600,대치동
3,,청담대림이-편한세상,11680-98,0030,0033,2002,개인,,,"220,000",...,00027,00000,4166819,01,11680,0,11680,개인,10400,청담동
4,,강남한양수자인(4단지),11680-4298,0686,0000,2014,개인,,,"159,000",...,00260,00000,3122014,02,11680,0,11680,개인,11200,자곡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761,,두산5,11260-3753,1522,0001,2000,개인,,,"63,000",...,00090,00000,4118361,01,11260,0,11260,개인,10100,면목동
4762,,사가정센트럴아이파크,11260-4104,1545,0000,2020,개인,,,"119,800",...,00040,00000,4118135,01,11260,0,11260,개인,10100,면목동
4763,,신도브래뉴,11260-149,0169,0010,2004,개인,,,"48,000",...,00946,00000,3000001,04,11260,0,11260,개인,10400,묵동
4764,,브라운스톤태릉,11260-108,0387,0000,2004,개인,,,"87,800",...,00059,00000,4118047,01,11260,0,11260,개인,10400,묵동


In [52]:
real_estate_schema = {
    "sggCd": "법정동시군구코드",
    "umdCd": "법정동읍면동코드",
    "landCd": "법정동지번코드",
    "bonbun": "법정동본번코드",
    "bubun": "법정동부번코드",
    "roadNm": "도로명",
    "roadNmSggCd": "도로명시군구코드",
    "roadNmCd": "도로명코드",
    "roadNmSeq": "도로명일련번호코드",
    "roadNmbCd": "도로명지상지하코드",
    "roadNmBonbun": "도로명건물본번호코드",
    "roadNmBubun": "도로명건물부번호코드",
    "umdNm": "법정동",
    "aptNm": "아파트명",
    "jibun": "지번",
    "excluUseAr": "전용면적",
    "dealYear": "계약년도",
    "dealMonth": "계약월",
    "dealDay": "계약일",
    "dealAmount": "거래금액(만원)",
    "floor": "층",
    "buildYear": "건축년도",
    "aptSeq": "단지 일련번호",
    "cdealType": "해제여부",
    "cdealDay": "해제사유발생일",
    "dealingGbn": "거래유형(중개 및 직거래 여부)",
    "estateAgentSggNm": "중개사소재지(시군구 단위)",
    "rgstDate": "등기일자",
    "aptDong": "아파트 동명",
    "slerGbn": "거래주체정보_매도자(개인/법인/공공기관/기타)",
    "buyerGbn": "거래주체정보_매수자(개인/법인/공공기관/기타)",
    "landLeaseholdGbn": "토지임대부 아파트 여부"
}


In [53]:
df_seoul_real_estate_data.rename(columns=real_estate_schema, inplace=True)
df_seoul_real_estate_data.head()

,아파트 동명,아파트명,단지 일련번호,법정동본번코드,법정동부번코드,건축년도,거래주체정보_매수자(개인/법인/공공기관/기타),해제사유발생일,해제여부,거래금액(만원),...,도로명건물본번호코드,도로명건물부번호코드,도로명코드,도로명일련번호코드,도로명시군구코드,도로명지상지하코드,법정동시군구코드,거래주체정보_매도자(개인/법인/공공기관/기타),법정동읍면동코드,법정동
0,,개나리래미안,11680-3680,0754,0000,2006,개인,,,"322,000",...,00306,00000,3122008,01,11680,0,11680,개인,10100,역삼동
1,,개포2차현대아파트(220),11680-371,0655,0002,1988,개인,,,"136,500",...,00103,00000,3005086,02,11680,0,11680,개인,10300,개포동
2,,대치삼성,11680-205,1014,0003,2000,개인,,,"290,000",...,00022,00000,4166245,01,11680,0,11680,개인,10600,대치동
3,,청담대림이-편한세상,11680-98,0030,0033,2002,개인,,,"220,000",...,00027,00000,4166819,01,11680,0,11680,개인,10400,청담동
4,,강남한양수자인(4단지),11680-4298,0686,0000,2014,개인,,,"159,000",...,00260,00000,3122014,02,11680,0,11680,개인,11200,자곡동


In [54]:
df_seoul_real_estate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4766 entries, 0 to 4765
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   아파트 동명                     4766 non-null   object
 1   아파트명                       4766 non-null   object
 2   단지 일련번호                    4766 non-null   object
 3   법정동본번코드                    4766 non-null   object
 4   법정동부번코드                    4766 non-null   object
 5   건축년도                       4766 non-null   object
 6   거래주체정보_매수자(개인/법인/공공기관/기타)  4766 non-null   object
 7   해제사유발생일                    4766 non-null   object
 8   해제여부                       4766 non-null   object
 9   거래금액(만원)                   4766 non-null   object
 10  계약일                        4766 non-null   object
 11  계약월                        4766 non-null   object
 12  계약년도                       4766 non-null   object
 13  거래유형(중개 및 직거래 여부)          4766 non-null   object
 14  중개사소재지(시

In [55]:
# 거래금액 칼럼 데이터타입 변환환
df_seoul_real_estate_data['거래금액(만원)'] = df_seoul_real_estate_data['거래금액(만원)'].str.replace(",", "").astype(int)

In [21]:
# 거래금액을 평균 값으로 그룹화
grouped_seoul_2020 = df_seoul_real_estate_data.groupby(["법정동"], as_index=False).agg({"거래금액(만원)": "mean"})
grouped_seoul_2020

,법정동,거래금액(만원)
0,가락동,128026.271186
1,가산동,25238.666667
2,가양동,74722.666667
3,갈현동,49490.000000
4,강일동,97096.000000
...,...,...
268,효창동,104550.000000
269,후암동,41000.000000
270,휘경동,70476.666667
271,흑석동,156169.444444


In [28]:
# 거래금액을 평균 값으로 그룹화
grouped_seoul_2021 = df_seoul_real_estate_data.groupby(["법정동"], as_index=False).agg({"거래금액(만원)": "mean"})
grouped_seoul_2021

,법정동,거래금액(만원)
0,가락동,168000.000000
1,가산동,21620.000000
2,가양동,109275.000000
3,갈현동,46500.000000
4,강일동,112666.666667
...,...,...
210,회현동1가,110000.000000
211,효제동,38933.333333
212,효창동,150500.000000
213,휘경동,36966.666667


In [35]:
grouped_seoul_2022 = df_seoul_real_estate_data.groupby(["법정동"], as_index=False).agg({"거래금액(만원)": "mean"})
grouped_seoul_2022

,법정동,거래금액(만원)
0,가락동,127614.285714
1,가산동,15125.000000
2,가양동,91766.666667
3,갈현동,37000.000000
4,강일동,71125.000000
...,...,...
167,회기동,21500.000000
168,효제동,18000.000000
169,후암동,105000.000000
170,휘경동,38900.000000


In [42]:
grouped_seoul_2023 = df_seoul_real_estate_data.groupby(["법정동"], as_index=False).agg({"거래금액(만원)": "mean"})
grouped_seoul_2023

,법정동,거래금액(만원)
0,가락동,163600.000000
1,가산동,32680.000000
2,가양동,76400.000000
3,갈현동,54250.000000
4,강일동,85000.000000
...,...,...
216,효제동,16000.000000
217,효창동,139000.000000
218,휘경동,41938.888889
219,흑석동,121730.000000


In [49]:
grouped_seoul_2024 = df_seoul_real_estate_data.groupby(["법정동"], as_index=False).agg({"거래금액(만원)": "mean"})
grouped_seoul_2024

,법정동,거래금액(만원)
0,가락동,156911.904762
1,가산동,68000.000000
2,가양동,80108.695652
3,갈현동,45143.000000
4,강일동,92000.000000
...,...,...
228,효창동,142250.000000
229,후암동,88750.000000
230,휘경동,60327.500000
231,흑석동,150203.125000


In [56]:
grouped_seoul_2025 = df_seoul_real_estate_data.groupby(["법정동"], as_index=False).agg({"거래금액(만원)": "mean"})
grouped_seoul_2025

,법정동,거래금액(만원)
0,가락동,168680.921053
1,가산동,40060.000000
2,가양동,75619.230769
3,갈현동,61483.333333
4,강일동,88550.000000
...,...,...
245,효창동,148625.000000
246,후암동,184000.000000
247,휘경동,66906.666667
248,흑석동,188842.105263


In [77]:
seoul = pd.merge(grouped_seoul_2020, grouped_seoul_2021, on='법정동', how='outer', suffixes=('_2020', '_2021'))
seoul

,법정동,거래금액(만원)_2020,거래금액(만원)_2021
0,가락동,128026.271186,168000.000000
1,가산동,25238.666667,21620.000000
2,가양동,74722.666667,109275.000000
3,갈현동,49490.000000,46500.000000
4,강일동,97096.000000,112666.666667
...,...,...,...
276,효창동,104550.000000,150500.000000
277,후암동,41000.000000,NaN
278,휘경동,70476.666667,36966.666667
279,흑석동,156169.444444,195750.000000


In [78]:
seoul = pd.merge(seoul, grouped_seoul_2022, on='법정동', how='outer', suffixes=('_2021', '_2022'))
seoul

,법정동,거래금액(만원)_2020,거래금액(만원)_2021,거래금액(만원)
0,가락동,128026.271186,168000.000000,127614.285714
1,가산동,25238.666667,21620.000000,15125.000000
2,가양동,74722.666667,109275.000000,91766.666667
3,갈현동,49490.000000,46500.000000,37000.000000
4,강일동,97096.000000,112666.666667,71125.000000
...,...,...,...,...
278,효창동,104550.000000,150500.000000,NaN
279,후암동,41000.000000,NaN,105000.000000
280,휘경동,70476.666667,36966.666667,38900.000000
281,흑석동,156169.444444,195750.000000,165000.000000


In [79]:
seoul = pd.merge(seoul, grouped_seoul_2023, on='법정동', how='outer', suffixes=('_2022', '_2023'))
seoul

,법정동,거래금액(만원)_2020,거래금액(만원)_2021,거래금액(만원)_2022,거래금액(만원)_2023
0,가락동,128026.271186,168000.000000,127614.285714,163600.000000
1,가산동,25238.666667,21620.000000,15125.000000,32680.000000
2,가양동,74722.666667,109275.000000,91766.666667,76400.000000
3,갈현동,49490.000000,46500.000000,37000.000000,54250.000000
4,강일동,97096.000000,112666.666667,71125.000000,85000.000000
...,...,...,...,...,...
283,효창동,104550.000000,150500.000000,NaN,139000.000000
284,후암동,41000.000000,NaN,105000.000000,NaN
285,휘경동,70476.666667,36966.666667,38900.000000,41938.888889
286,흑석동,156169.444444,195750.000000,165000.000000,121730.000000


In [80]:
seoul = pd.merge(seoul, grouped_seoul_2024, on='법정동', how='outer', suffixes=('_2023', '_2024'))
seoul

,법정동,거래금액(만원)_2020,거래금액(만원)_2021,거래금액(만원)_2022,거래금액(만원)_2023,거래금액(만원)
0,가락동,128026.271186,168000.000000,127614.285714,163600.000000,156911.904762
1,가산동,25238.666667,21620.000000,15125.000000,32680.000000,68000.000000
2,가양동,74722.666667,109275.000000,91766.666667,76400.000000,80108.695652
3,갈현동,49490.000000,46500.000000,37000.000000,54250.000000,45143.000000
4,강일동,97096.000000,112666.666667,71125.000000,85000.000000,92000.000000
...,...,...,...,...,...,...
290,효창동,104550.000000,150500.000000,NaN,139000.000000,142250.000000
291,후암동,41000.000000,NaN,105000.000000,NaN,88750.000000
292,휘경동,70476.666667,36966.666667,38900.000000,41938.888889,60327.500000
293,흑석동,156169.444444,195750.000000,165000.000000,121730.000000,150203.125000


In [81]:
seoul = pd.merge(seoul, grouped_seoul_2025, on='법정동', how='outer', suffixes=('_2024', '_2025'))
seoul

,법정동,거래금액(만원)_2020,거래금액(만원)_2021,거래금액(만원)_2022,거래금액(만원)_2023,거래금액(만원)_2024,거래금액(만원)_2025
0,가락동,128026.271186,168000.000000,127614.285714,163600.000000,156911.904762,168680.921053
1,가산동,25238.666667,21620.000000,15125.000000,32680.000000,68000.000000,40060.000000
2,가양동,74722.666667,109275.000000,91766.666667,76400.000000,80108.695652,75619.230769
3,갈현동,49490.000000,46500.000000,37000.000000,54250.000000,45143.000000,61483.333333
4,강일동,97096.000000,112666.666667,71125.000000,85000.000000,92000.000000,88550.000000
...,...,...,...,...,...,...,...
293,효창동,104550.000000,150500.000000,NaN,139000.000000,142250.000000,148625.000000
294,후암동,41000.000000,NaN,105000.000000,NaN,88750.000000,184000.000000
295,휘경동,70476.666667,36966.666667,38900.000000,41938.888889,60327.500000,66906.666667
296,흑석동,156169.444444,195750.000000,165000.000000,121730.000000,150203.125000,188842.105263
